** Build Adjacency Matrix **

In [1]:
import sqlite3
import json

In [2]:
DATABASE = "data.sqlite"

In [3]:
conn = sqlite3.connect(DATABASE)
cursor = conn.cursor()

Queries

In [4]:
# For getting the maximum row id
QUERY_MAX_ID = "SELECT id FROM interactions ORDER BY id DESC LIMIT 1"

# Get interaction data
QUERY_INTERACTION = "SELECT geneids1, geneids2, probability FROM interactions WHERE id = {}"

In [5]:
max_id = cursor.execute(QUERY_MAX_ID).fetchone()[0]

Step through every interaction.

1. If geneids1 not in matrix - insert it as dict.
2. If geneids2 not in matrix[geneids1] - insert it as []
3. If probability not in matrix[geneids1][geneids2] - insert it.
4. Perform the reverse.

In [6]:
matrix = {}
row_id = 0

while row_id <= max_id:
    row_id+= 1
    
    row = cursor.execute(QUERY_INTERACTION.format(row_id))
    row = row.fetchone()
    
    if row == None:
        continue
     
    id1 = row[0]
    id2 = row[1]
    prob = int(round(row[2],2) * 1000)

    # Forward
    if id1 not in matrix:
        matrix[id1] = {}

    if id2 not in matrix[id1]:
        matrix[id1][id2] = []

    if prob not in matrix[id1][id2]:
        matrix[id1][id2].append(prob)
    
    # Backwards
    if id2 not in matrix:
        matrix[id2] = {}

    if id1 not in matrix[id2]:
        matrix[id2][id1] = []

    if prob not in matrix[id2][id1]:
        matrix[id2][id1].append(prob)
    

In [7]:
with open("matrix.json", "w+") as file:
    file.write(json.dumps( matrix ))